# DX 704 Week 9 Project

This week's project will build an email spam classifier based on the Enron email data set.
You will perform your own feature extraction, and use naive Bayes to estimate the probability that a particular email is spam or not.
Finally, you will review the tradeoffs from different thresholds for automatically sending emails to the junk folder.

The full project description and a template notebook are available on GitHub: [Project 9 Materials](https://github.com/bu-cds-dx704/dx704-project-09).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download Data Set

We will be using the Enron spam data set as prepared in this GitHub repository.

https://github.com/MWiechmann/enron_spam_data

You may need to download this differently depending on your environment.

In [66]:
!wget https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip

--2025-11-02 19:15:57--  https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip [following]
--2025-11-02 19:15:57--  https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15642124 (15M) [application/zip]
Saving to: ‘enron_spam_data.zip’

enron_spam_data.zip 100%[===================>]  14.92M  --.-KB/s    in 0.1s    

2025-11-02 19:15:57 (133 MB/s) - ‘enron_spam

In [67]:
import pandas as pd

In [68]:
# pandas can read the zip file directly
enron_spam_data = pd.read_csv("enron_spam_data.zip")
enron_spam_data

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14
...,...,...,...,...,...
33711,33711,= ? iso - 8859 - 1 ? q ? good _ news _ c = eda...,"hello , welcome to gigapharm onlinne shop .\np...",spam,2005-07-29
33712,33712,all prescript medicines are on special . to be...,i got it earlier than expected and it was wrap...,spam,2005-07-29
33713,33713,the next generation online pharmacy .,are you ready to rock on ? let the man in you ...,spam,2005-07-30
33714,33714,bloow in 5 - 10 times the time,learn how to last 5 - 10 times longer in\nbed ...,spam,2005-07-30


In [69]:
(enron_spam_data["Spam/Ham"] == "spam").mean()

np.float64(0.5092834262664611)

## Part 2: Design a Feature Extractor

Design a feature extractor for this data set and write out two files of features based on the text.
Don't forget that both the Subject and Message columns are relevant sources of text data.
For each email, you should count the number of repetitions of each feature present.
The auto-grader will assume that you are using a multinomial distribution in the following problems.

In [70]:
# YOUR CHANGES HERE

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

all_texts = (
    enron_spam_data['Subject'].fillna('') + ' ' +
    enron_spam_data['Message'].fillna('')
)

labels = (enron_spam_data['Spam/Ham'].str.lower() == 'spam').astype(int)

vectorizer = CountVectorizer(lowercase=True, token_pattern=r'\b[a-z0-9]+\b', min_df=5)
X = vectorizer.fit_transform(all_texts)  

print(f"Feature matrix shape: {X.shape}")
print(f"Sparsity: {(1 - X.nnz / (X.shape[0] * X.shape[1])):.2%}")

from scipy.sparse import save_npz
save_npz("features_sparse.npz", X)
labels_df = pd.DataFrame({'label': labels})
labels_df.to_csv("labels.csv", index=False)

vocab_df = pd.DataFrame({'feature': vectorizer.get_feature_names_out()})
vocab_df.to_csv("vocabulary.csv", index=False)

print("Files created:")
print("  - features_sparse.npz (sparse format)")
print("  - labels.csv")
print("  - vocabulary.csv")


Feature matrix shape: (33716, 44120)
Sparsity: 99.73%
Files created:
  - features_sparse.npz (sparse format)
  - labels.csv
  - vocabulary.csv


Assign a row to the test data set if `Message ID % 30 == 0` and assign it to the training data set otherwise.
Write two files, "train-features.tsv" and "test-features.tsv" with two columns, Message ID and features_json.
The features_json column should contain a JSON dictionary where the keys are your feature names and the values are integer feature values.
This will give us a sparse feature representation.


In [71]:
# YOUR CHANGES HERE
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import json

all_texts = (enron_spam_data['Subject'].fillna('') + ' ' + enron_spam_data['Message'].fillna(''))

vectorizer = CountVectorizer(lowercase=True, token_pattern=r'\b[a-z0-9]+\b', min_df=5)
X = vectorizer.fit_transform(all_texts)
feature_names = vectorizer.get_feature_names_out()

message_ids = enron_spam_data.index.values
labels = (enron_spam_data['Spam/Ham'].str.lower() == 'spam').astype(int)  # 1 = spam, 0 = ham

test_mask = (message_ids % 30) == 0
train_mask = ~test_mask

def sparse_row_to_json(row_idx, sparse_matrix, feature_names):
    row = sparse_matrix[row_idx]
    non_zero_indices = row.nonzero()[1]
    non_zero_values = row.data
    feature_dict = {feature_names[idx]: int(val) for idx, val in zip(non_zero_indices, non_zero_values)}
    return json.dumps(feature_dict)

train_data = []
for i in range(len(message_ids)):
    if train_mask[i]:
        train_data.append({
            'Message ID': message_ids[i],
            'features_json': sparse_row_to_json(i, X, feature_names),
            'label': labels.iloc[i]
        })
pd.DataFrame(train_data).to_csv('train-features.tsv', sep='\t', index=False)

test_data = []
for i in range(len(message_ids)):
    if test_mask[i]:
        test_data.append({
            'Message ID': message_ids[i],
            'features_json': sparse_row_to_json(i, X, feature_names),
            'label': labels.iloc[i]
        })
pd.DataFrame(test_data).to_csv('test-features.tsv', sep='\t', index=False)


Submit "train-features.tsv" and "test-features.tsv" in Gradescope.

Hint: these features will be graded based on the test accuracy of a logistic regression based on the training features.
This is to make sure that your feature set is not degenerate; you do not need to compute this regression yourself.
You can separately assess your feature quality based on your results in part 6.

## Part 3: Compute Conditional Probabilities

Based on your training data, compute appropriate conditional probabilities for use with naïve Bayes.
Use of additive smoothing with $\alpha=1$ to avoid zeros.


In [72]:
# YOUR CHANGES HERE

train_data = pd.read_csv('train-features.tsv', sep='\t')
train_data['features_json'] = train_data['features_json'].fillna('{}')

all_features = set()
feature_counts_spam = {}
feature_counts_ham = {}

for _, row in train_data.iterrows():
    features = json.loads(row['features_json'])
    is_spam = row['label'] == 1
    for feature, count in features.items():
        all_features.add(feature)
        if is_spam:
            feature_counts_spam[feature] = feature_counts_spam.get(feature, 0) + count
        else:
            feature_counts_ham[feature] = feature_counts_ham.get(feature, 0) + count

vocab = sorted(all_features)
V = len(vocab)
total_spam = sum(feature_counts_spam.values())
total_ham = sum(feature_counts_ham.values())
alpha = 1

prob_data = []
for word in vocab:
    count_spam = feature_counts_spam.get(word, 0)
    count_ham = feature_counts_ham.get(word, 0)
    spam_prob = (count_spam + alpha) / (total_spam + alpha * V)
    ham_prob = (count_ham + alpha) / (total_ham + alpha * V)
    prob_data.append({'feature': word, 'ham_probability': ham_prob, 'spam_probability': spam_prob})


Save the conditional probabilities in a file "feature-probabilities.tsv" with columns feature, ham_probability and spam_probability.

In [73]:
# YOUR CHANGES HERE

prob_df = pd.DataFrame(prob_data)
prob_df.to_csv('feature-probabilities.tsv', sep='\t', index=False)


Submit "feature-probabilities.tsv" in Gradescope.

## Part 4: Implement a Naïve Bayes Classifier

Implement a naïve Bayes classifier based on your previous feature probabilities.

In [81]:
# YOUR CHANGES HERE

from collections import Counter

alpha = 1

train_data = pd.read_csv('train-features.tsv', sep='\t')
train_data['features_json'] = train_data['features_json'].fillna('{}')

spam_emails = train_data[train_data['label'] == 1]
ham_emails = train_data[train_data['label'] == 0]

spam_counts = Counter()
ham_counts = Counter()
vocab = set()

for features_json in spam_emails['features_json']:
    f = json.loads(features_json)
    spam_counts.update(f)
    vocab.update(f.keys())

for features_json in ham_emails['features_json']:
    f = json.loads(features_json)
    ham_counts.update(f)
    vocab.update(f.keys())

vocab = sorted(vocab)
V = len(vocab)

total_spam = sum(spam_counts.values())
total_ham = sum(ham_counts.values())

n_total = len(train_data)
n_spam = len(spam_emails)
n_ham = len(ham_emails)
log_prior_spam = np.log(n_spam / n_total)
log_prior_ham = np.log(n_ham / n_total)

log_spam_probs = {word: np.log((spam_counts.get(word, 0) + alpha) / (total_spam + alpha * V)) for word in vocab}
log_ham_probs = {word: np.log((ham_counts.get(word, 0) + alpha) / (total_ham + alpha * V)) for word in vocab}

def predict_nb(df, log_spam_probs, log_ham_probs, log_prior_spam, log_prior_ham, vocab, alpha=1):
    ham_probs = []
    spam_probs = []
    V = len(vocab)
    log_min_spam = np.log(alpha / (total_spam + alpha * V))
    log_min_ham = np.log(alpha / (total_ham + alpha * V))

    for features_json in df['features_json']:
        features = json.loads(features_json)
        log_prob_spam = log_prior_spam
        log_prob_ham = log_prior_ham
        for word, count in features.items():
            log_prob_spam += count * log_spam_probs.get(word, log_min_spam)
            log_prob_ham += count * log_ham_probs.get(word, log_min_ham)
        max_log = max(log_prob_spam, log_prob_ham)
        prob_spam = np.exp(log_prob_spam - max_log)
        prob_ham = np.exp(log_prob_ham - max_log)
        total = prob_spam + prob_ham
        spam_probs.append(prob_spam / total)
        ham_probs.append(prob_ham / total)
    return ham_probs, spam_probs


Save your prediction probabilities to "train-predictions.tsv" with columns Message ID, ham and spam.

In [82]:
# YOUR CHANGES HERE



ham_probs_train, spam_probs_train = predict_nb(train_data, log_spam_probs, log_ham_probs, log_prior_spam, log_prior_ham, vocab)
train_data['ham'] = ham_probs_train
train_data['spam'] = spam_probs_train
train_data[['Message ID', 'ham', 'spam']].to_csv('train-predictions.tsv', sep='\t', index=False)

test_data = pd.read_csv('test-features.tsv', sep='\t')
test_data['features_json'] = test_data['features_json'].fillna('{}')
ham_probs_test, spam_probs_test = predict_nb(test_data, log_spam_probs, log_ham_probs, log_prior_spam, log_prior_ham, vocab)
test_data['ham'] = ham_probs_test
test_data['spam'] = spam_probs_test
test_data[['Message ID', 'ham', 'spam']].to_csv('test-predictions.tsv', sep='\t', index=False)



Submit "train-predictions.tsv" in Gradescope.

## Part 5: Predict Spam Probability for Test Data

Use your previous classifier to predict spam probability for the test data.

In [76]:
# YOUR CHANGES HERE


feature_probs = pd.read_csv('feature-probabilities.tsv', sep='\t')
feature_probs.set_index('feature', inplace=True)
log_ham_dict = np.log(feature_probs['ham_probability']).to_dict()
log_spam_dict = np.log(feature_probs['spam_probability']).to_dict()

test_data = pd.read_csv('test-features.tsv', sep='\t')
test_data['features_json'] = test_data['features_json'].fillna('{}')

train_data = pd.read_csv('train-features.tsv', sep='\t')
n_total = len(train_data)
n_spam = (train_data['label'] == 1).sum()
n_ham = (train_data['label'] == 0).sum()
log_prior_spam = np.log(n_spam / n_total)
log_prior_ham = np.log(n_ham / n_total)

ham_probs = []
spam_probs = []

for features_json in test_data['features_json']:
    features = json.loads(features_json)
    log_prob_spam = log_prior_spam
    log_prob_ham = log_prior_ham
    for word, count in features.items():
        log_prob_spam += count * log_spam_dict.get(word, 0)
        log_prob_ham += count * log_ham_dict.get(word, 0)
    max_log = max(log_prob_spam, log_prob_ham)
    prob_spam = np.exp(log_prob_spam - max_log)
    prob_ham = np.exp(log_prob_ham - max_log)
    total = prob_spam + prob_ham
    ham_probs.append(prob_ham / total)
    spam_probs.append(prob_spam / total)

test_data['ham'] = ham_probs
test_data['spam'] = spam_probs


Save your prediction probabilities in "test-predictions.tsv" with the same columns as "train-predictions.tsv".

In [77]:
# YOUR CHANGES HERE

test_data[['Message ID', 'ham', 'spam']].to_csv('test-predictions.tsv', sep='\t', index=False)



Submit "test-predictions.tsv" in Gradescope.

## Part 6: Construct ROC Curve

For every probability threshold from 0.01 to .99 in increments of 0.01, compute the false and true positive rates from the test data using the spam class for positives.
That is, if the predicted spam probability is greater than or equal to the threshold, predict spam.

In [78]:
# YOUR CHANGES HERE

test_preds = pd.read_csv('test-predictions.tsv', sep='\t')
test_features = pd.read_csv('test-features.tsv', sep='\t')

test_data = test_preds.merge(test_features[['Message ID', 'label']], on='Message ID', how='inner')

y_true = test_data['label'].values
y_score = test_data['spam'].values  
thresholds = np.arange(0.01, 1.00, 0.01)
roc_list = []

for t in thresholds:
    y_pred = (y_score >= t).astype(int)
    tp = np.sum((y_pred == 1) & (y_true == 1))
    fp = np.sum((y_pred == 1) & (y_true == 0))
    fn = np.sum((y_pred == 0) & (y_true == 1))
    tn = np.sum((y_pred == 0) & (y_true == 0))
    
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    roc_list.append({'threshold': t, 'false_positive_rate': fpr, 'true_positive_rate': tpr})

roc_df = pd.DataFrame(roc_list)


Save this data in a file "roc.tsv" with columns threshold, false_positive_rate and true_positive rate.

In [79]:
# YOUR CHANGES HERE

roc_df.to_csv('roc.tsv', sep='\t', index=False)


Submit "roc.tsv" in Gradescope.

## Part 7: Signup for Gemini API Key

Create a free Gemini API key at https://aistudio.google.com/app/api-keys.
You will need to do this with a personal Google account - it will not work with your BU Google account.
This will not incur any charges unless you configure billing information for the key.

You will be asked to start a Gemini free trial for week 11.
This will not incur any charges unless you exceed expected usage by an order of magnitude.


No submission needed.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.